In [ ]:
# import google.generativeai as genai

# # Set your API key as an environment variable for security
# os.environ['GOOGLE_API_KEY'] = '---API KEY---'

# # Configure the API with your key
# genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

# prompt_template = """
#                         You are an expert at analyzing social media content.
#                         I will provide you with a tweet in Thai mainly about bread or bakery.
#                         Your task is to analyze the content and return a JSON object with the following keys:
#                         1. "summary": A very short summary of the tweet's main topic in Thai.
#                         2. "sentiment": The sentiment of the tweet. The value should be one of "positive", "negative", or "neutral".
#                         3. "tags": A list of up to 5 relevant keywords or tags from the content in Thai.

#                         Example output format (JSON):
#                         {{
#                         "summary": "สรุปสั้นๆ เกี่ยวกับเนื้อหา",
#                         "sentiment": "positive",
#                         "tags": ["คีย์เวิร์ด", "คำสำคัญ"]
#                         }}

#                         Return ONLY the JSON object, without any other text or Markdown formatting.

#                         Now, analyze the following tweet:
#                         {tweet_content}
#                         """
# model = genai.GenerativeModel('gemini-2.0-flash-lite')
# tweet_data = tweet
# formatted_prompt = prompt_template.format(tweet_content=tweet_data)
# response = model.generate_content(formatted_prompt)

In [33]:
import os
import re
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display
import google.generativeai as genai

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Set your API key as an environment variable for security
os.environ['GOOGLE_API_KEY'] = os.getenv("GEMINI_API_KEY")

# Configure the API with your key
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

prompt_template = """Can you help me name all competitor of CP Group = CPAll CPF CPXTRA?
                     For example "เบทาโกร"' "ไทยยูเนี่ยน"' "คาร์กิลล์"' "สหฟาร์ม"' "CJ"' "ฟาร์มเฮ้าส์"' "Go Wholesale"'
                    """
model = genai.GenerativeModel('gemini-2.0-flash-lite')
#tweet_data = tweet
#formatted_prompt = prompt_template.format(tweet_content=tweet_data)
response = model.generate_content(prompt_template)

E0000 00:00:1759373768.045951  499574 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [35]:
response.text

"Okay, I can help you brainstorm competitors for the CP Group subsidiaries: CPAll (convenience stores), CPF (food and agriculture), and CPXTRA (wholesale).  Here's a breakdown with examples, considering the different business segments:\n\n**1. CPAll (7-Eleven): Convenience Store Competitors**\n\n*   **Major Competitors:**\n    *   **FamilyMart:** (Presence in Thailand) - One of the biggest international competitors, often with similar product ranges and services.\n    *   **Tops Daily:** (Owned by Central Retail) - Growing chain of convenience stores.\n    *   **Mini Big C:** (Owned by Big C) -  Also a significant player, especially strong in urban areas.\n    *   **Lotus's Express:** (Owned by CP Group previously) - Competitors with own convenience store format.\n*   **Smaller/Regional Competitors:**\n    *   Local convenience stores:  A large number of independent convenience stores across Thailand.\n    *   Gas station convenience stores: (e.g., those associated with PTT stations)\n

In [ ]:
# # Put your Gemini response here
# text = response.text

# # Step 1: Split into business sections (CPAll, CPF, CPXTRA, TRUE)
# business_blocks = re.split(r"\*\*\d+\.\s+", text)
# business_blocks = [b.strip() for b in business_blocks if b.strip()]

# rows = []

# for block in business_blocks:
#     # Extract business name (e.g., "CPAll (Retail - Convenience Stores...)")
#     business_match = re.match(r"([A-Z]+[A-Za-z0-9 ]*)", block)
#     if not business_match:
#         continue
#     business = business_match.group(1).strip()

#     # Step 2: Extract categories and competitors
#     # Categories start with **Category:** and competitors with "*   "
#     category = None
#     for line in block.splitlines():
#         line = line.strip()

#         # Category line (e.g., **Convenience Stores:**)
#         if line.startswith("*") and "**" in line:
#             category = re.sub(r"[*:]", "", line).strip()
#             continue

#         # Competitors (lines starting with * or indented bullets)
#         if line.startswith("*"):
#             comp = re.sub(r"[*]", "", line).strip()
#             if comp and category:
#                 rows.append({"Business": business, "Category": category, "Competitor": comp})

# # Step 3: Convert to DataFrame
# df = pd.DataFrame(rows)

# # Clean up text (remove bold ** and extra spaces)
# df["Category"] = df["Category"].str.replace(r"\*\*", "", regex=True).str.strip()
# df["Competitor"] = df["Competitor"].str.replace(r"\*\*", "", regex=True).str.strip()


In [ ]:
# #df
# #display(df)
# pd.set_option("display.max_rows", None)   # show all rows
# pd.set_option("display.max_columns", None) # show all columns
# pd.set_option("display.width", None)      # don't wrap columns
# pd.set_option("display.max_colwidth", None) # show full text

In [32]:
# df

In [40]:
text = response.text  # your Gemini output

rows = []
business = None
category = None

for line in text.splitlines():
    line = line.strip()

    # Business sections like **1. CPAll (Retail...)**
    if re.match(r"\*\*\d+\.", line):
        business = re.sub(r"\*\*|\d+\.|\(.*?\)", "", line).strip()
        continue

    # Category sections like *   **Convenience Stores:**
    if line.startswith("*") and "**" in line and ":" in line:
        category = re.sub(r"[*:]", "", line).replace("**", "").strip()
        continue

    # Competitors: lines starting with '*' but not categories
    if line.startswith("*"):
        competitor = re.sub(r"^\*+", "", line).strip()
        if competitor and business and category:
            rows.append({
                "Business": business,
                "Category": category,
                "Competitor": competitor
            })

df = pd.DataFrame(rows)

# Clean up text
df["Business"] = df["Business"].str.replace(r"\*\*", "", regex=True).str.strip()
df["Category"] = df["Category"].str.replace(r"\*\*", "", regex=True).str.strip()
df["Competitor"] = df["Competitor"].str.replace(r"\*\*", "", regex=True).str.strip()

In [42]:
# Show full DataFrame (no truncation)
pd.set_option("display.max_rows", None)
#df

In [45]:
# 1. Clean Business column (remove ":" at the end and strip spaces)
df["Business"] = df["Business"].str.replace(r":$", "", regex=True).str.strip()

# 2. Clean Category & Competitor (remove anything in parentheses and extra spaces)
df["Category"] = df["Category"].str.replace(r"\(.*?\)", "", regex=True).str.strip()
df["Competitor"] = df["Competitor"].str.replace(r"\(.*?\)", "", regex=True).str.strip()

# 3. Manually correct CPXTRA’s category and Clean up Business column
df.loc[df["Business"]=="CPXTRA", "Category"] = "Wholesale"
df["Business"] = df["Business"].str.split("-").str[0].str.strip()

# Optional: reset index for nice ordering
#df = df.reset_index(drop=True)
df

,Business,Category,Competitor
0,CPAll,Convenience Stores,Tops Daily
1,CPAll,Convenience Stores,FamilyMart
2,CPAll,Convenience Stores,Mini Big C
3,CPAll,Convenience Stores,Lotus's Go Fresh
4,CPAll,Convenience Stores,Lawson 108
5,CPAll,Convenience Stores,AMAZON
6,CPAll,Supermarkets/Hypermarkets,Lotus's
7,CPAll,Supermarkets/Hypermarkets,Big C
8,CPAll,Supermarkets/Hypermarkets,Tops Supermarket/Food Hall/Central Food Hall
9,CPAll,Supermarkets/Hypermarkets,Villa Market
